# Lab 5
Owen Goodwin (500909196)

10/08/19

In [1]:
from hashing import *

H = HashTable()
H[54] = "cat"
H[26] = "dog"
H[93] = "lion"
H[17] = "tiger"
H[77] = "bird"
H[31] = "cow"
H[44] = "goat"
H[55] = "pig"
H[20] = "chicken"
print(H.data)

print(H[20])

H[20] = 'duck'

print(H[20])

print(H.data)

['goat', None, None, None, 'dog', 'lion', None, None, None, 'cow', 'cat', 'bird', 'pig', None, None, None, None, 'tiger', None, None, 'chicken', None]
chicken
duck
['goat', None, None, None, 'dog', 'lion', None, None, None, 'cow', 'cat', 'bird', 'pig', None, None, None, None, 'tiger', None, None, 'duck', None]


**1. Why did you choose a particular resizing method? How much does the table grow?**
To resize the table, I take a copy of the current table's keys and data, and initialize a new one twice as long, and then recalculate the new positions of the old values. I chose to resize the table this way simply because it seemed like the most natural method.

**2. What loading factor threshold have you used for the resizing the table and why that particular choice?**
The table is resized once the loading factor hits 70%. Through some testing, I've noticed that the number of collisions goes up sharply as the loading factor goes beyond 70%, so it seemed like resizing at 70% would be a good decision for performance.

### The improved ``put()`` method

In [2]:
def put(self, key, data):
    self.resize()#determine whether or not we need to resize the table
    hashvalue = self.hashfunction(key, len(self.slots))#get hash val with given key and length of slots

    if self.slots[hashvalue] == None:#if the slot is empty, add the thing
        self.slots[hashvalue] = key
        self.data[hashvalue] = data
    else:
        if self.slots[hashvalue] == key:#if the slot already has the same key, replace the data
            self.data[hashvalue] = data  #replace
        else:# else, rehash for a new slot
            nextslot = self.rehash(hashvalue, len(self.slots))
            while self.slots[nextslot] != None and self.slots[nextslot] != key:
                nextslot = self.rehash(nextslot, len(self.slots))

            if self.slots[nextslot] == None:
                self.slots[nextslot] = key
                self.data[nextslot] = data
            else:
                self.data[nextslot] = data #replace

To be honest, the only real difference is the call to ``resize()`` at the start, to decide whether we should resize the table. That's all we needed to do anyways. I did end up spending a good chunk of time trying to figure out why the value that triggered the resize ended up in the wrong spot while all others were fine, until I realised I had been determining the ``hashvalue`` before calling ``resize()``, effectively determining the new element's position in the old table, and putting it there in the new table. Swapping the first two lines fixed that real quick.

In [3]:
#Function to check if the table needs resizing, and if so, resizes the table
def resize(self):
    if self.getLoadFactor() > 0.7:# We resize if its over 70% full
        oldData = self.data #Take a copy of the pre-resize slots and data.
        oldSlots = self.slots
        self.data = [None] * (len(oldData) * 2)# Make new lists, twice as long as the old ones
        self.slots = [None] * (len(oldSlots) * 2)
        self.fix(oldSlots, oldData)# Take the old slots and data, and put them in the correct spots in the new table
        self.size = self.size * 2# Make sure we remember our resize

#Function to fix up the table after a resize, by calculating the new position of all the stuff
def fix(self, oldSlots, oldData):
    for i in range(0, len(oldSlots)):# Start going through all of our old stuff
        if oldSlots[i] != None:# We don't care if the slot is empty
            key = oldSlots[i] # Take the old key-val pair...
            val = oldData[i]
            self.put(key, val) # ...and put it in it's proper home in the bigger table

#Function to get the current load factor. This value is how we decide when to resize
def getLoadFactor(self):
    # Load factor = # of filled slots / # of slots total
    d = len(self.slots) # get the total number of slots
    n = 0 # create a counter var
    for slot in self.slots:# Go through each slot, and find out if it's filled. Add one if it is.
        if slot != None:
            n += 1
    #At this point, n is the number of slots that are filled.
    #print("There are " + str(d) + " slots, " + str(n) + " are full (" + str(n/d * 1.0) + ")")
    return n/d * 1.0# Calculate the load factor, as a float

These three methods are all part of the resizing process.

``resize()`` is called every time we call ``put()`` to add a new element. It checks the load factor and decides whether the table should be resized or not.

``getLoadFactor()`` is pretty self-explanitory, it calculates how many of the table's slots are occupied, and then divides that by the number of total slots to get the load factor.

``fix()`` is how we take the contents of the table before the resize and figure out their new position in the resized table, without this we'd have issues.